In [1]:
import meep as mp
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
from numpy import cos, sin
import json
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt

# create geometric and write it to a file

generate random numbers

In [2]:
def gen_part_size(num_crystal, size_crystal_base, weibull = True):
    a = 5. # shape of weibull distribution
    size_crystal_change = np.random.weibull(a, (num_crystal, 3))
    
    if weibull:
        size_crystal_base *= size_crystal_change
            
    size = []
    for i in range(num_crystal):
        size.append(mp.Vector3(*size_crystal[i, :]))
        
    return size

def gen_part_loc(num_crystal, size_solid = None, use_normal = False):
    if(size_solid == None):
        size_solid = [1, 1, 1]
    if use_normal:
        mean= (0, 0, 0)
        cov = [[0.1, 0, 0], [0, 0.1, 0], [0, 0, 0.1]]
        loc = np.random.multivariate_normal(mean, cov, (num_crystal))
    else:
        loc = np.random.uniform(-size_solid[0]/2, size_solid[0]/2, (num_crystal, 3))
    return loc

def gen_part_rot(num_crystal):
    theta = np.empty(num_crystal)
    for i in range(3):
        theta[i] = np.random.uniform(0, 2*np.pi, num_crystal)
    return theta

def gen_particle_geo(loc, theta_x, theta_y, theta_z):
    R = np.empty((num_crystal, 3, 3))

    Rx_matrix = np.empty((num_crystal, 3, 3))
    Ry_matrix = np.empty((num_crystal, 3, 3))
    Rz_matrix = np.empty((num_crystal, 3, 3))

    for i in range(num_crystal):
        Rx_matrix[i, :, :] = np.array([[1, 0, 0],
                       [0, cos(theta_x[i]), -sin(theta_x[i])], 
                      [0, sin(theta_x[i]), cos(theta_x[i])]])

        Ry_matrix[i, :, :] = np.array([[cos(theta_y[i]), 0, sin(theta_y[i])], 
                      [0, 1, 0],
                      [-sin(theta_y[i]), 0, cos(theta_y[i])]])

        Rz_matrix[i, :, :] = np.array([[cos(theta_z[i]), -sin(theta_z[i]), 0],
                     [sin(theta_z[i]), cos(theta_z[i]), 0],
                     [0, 0, 1]])

        R[i, :, :] = np.matmul(np.matmul(Ry_matrix[i, :, :], Rx_matrix[i, :, :]), Rz_matrix[i, :, :])


    og_x = np.array([[1, 0, 0] for i in range(num_crystal)])
    og_y = np.array([[0, 1, 0] for i in range(num_crystal)])
    og_z = np.array([[0, 0, 1] for i in range(num_crystal)])

    Rx_vector = np.empty((num_crystal, 3))
    Ry_vector = np.empty((num_crystal, 3))
    Rz_vector = np.empty((num_crystal, 3))

    for i in range(num_crystal):
        Rx_vector[i, :] = np.matmul(R[i, :, :], og_x[i, :])
        Ry_vector[i, :] = np.matmul(R[i, :, :], og_y[i, :])
        Rz_vector[i, :] = np.matmul(R[i, :, :], og_z[i, :])

    geometry = [solid_region,]

    for i in range(num_crystal):
        if (np.abs(loc[i, 0]) < size_solid[0] - size_crystal_base[0]/2 and 
        np.abs(loc[i, 1]) < size_solid[1] - size_crystal_base[1]/2 and 
        np.abs(loc[i, 2]) < size_solid[2] - size_crystal_base[2]/2):
            geometry.append(mp.Block(
                size_crystal[i],
                center = mp.Vector3(loc[i, 0], loc[i, 1], loc[i, 2]),
                e1 = Rx_vector[i, :],
                e2 = Ry_vector[i, :],
                e3 = Rz_vector[i, :],
                material=mp.Medium(epsilon=10.5)))
    return geometry

def out_para_geo(file_name, num_crystal, size_solid_l, size_crystal_l, loc, theta):
    to_write = [num_crystal, size_solid_l, size_crystal_l, loc, theta]
    for i in range(len(to_write)):
        if type(to_write[i]) is not int and type(to_write[i]) is not list:
            to_write[i] = to_write[i].tolist()
    with open(file_name, 'w') as f:
        json.dump(to_write, f)

#\\ad.monash.edu\home\User045\dche145\Documents\Abaqus\geometry_shapes
#\\Client\D$\source\working_with_meep

#output geometric files
def out_num_geo(file_name, range_geo=None, range_index = None):
    if range_index == None:
        range_index = [100, 100, 100]
    if range_geo == None:
        range_geo = [1.0, 1.0, 1.0]
    out_geo = np.zeros((range_index))
    for i in range(range_index[0]):
        for j in range(range_index[1]):
            for k in range(range_index[2]):
                coord = index2coord(np.array((i,j,k),dtype=float), range_index, range_geo)
                out_geo[i,j,k] = parts_ass[closest_node(coord, points)]
    with open(file_name, 'wb') as f:
        out_geo.tofile(f)

this is the function for outputting numerical geometry

In [13]:
class parts_ass_data:
    num_div = 20
    num_seed = num_div**3
    points = np.zeros((num_div**3, 3))

    p_range = np.array([1.0, 1.0, 1.0])
    p_div = p_range/num_div
    num_parts = 2
    eps_val = [1, 10]

    # vor = Voronoi(points)

    # assigning points and seeds

    def __init__(self, use_rand_ass = True, seed = 15):
        if use_rand_ass:
            self.random_ass(seed)
        else:
            self.checker_pattern()

    def random_ass(self, seed):
        np.random.seed(seed)
        self.points = np.random.rand(self.num_seed, 3) - 0.5
        parts_ass = np.random.randint(0, self.num_parts, (self.num_seed))
        self.parts_eps = [self.eps_val[parts_ass[i]] for i in range(self.num_seed)]

    def checker_pattern(self):
        for i in range(self.num_div):
            for j in range(self.num_div):
                for k in range(self.num_div):
                    index = [i,j,k]
                    self.points[index, :] = index2coord(index, [num_div,num_div,num_div], p_range)
        # this will produce checker pattern
        parts_ass = np.array([1 if i%2 else 0 for i in range(num_seed)])
        self.parts_eps = [self.eps_val[parts_ass[i]] for i in range(num_seed)]

psd = parts_ass_data()

def closest_node(node, nodes):
    # nodes = np.asarray(nodes)
    dist_2 = np.sum((nodes - node)**2, axis=1)
    return np.argmin(dist_2)

    
def my_mat(coord):
    if coord[0] <  0.25 and coord[0] > -0.25 :
        return mp.Medium(epsilon=10.5)
    else:
        return mp.Medium(epsilon=1)

def my_eps(coord):
    acoord = np.abs(coord)
    if (acoord[0] < 0.5 and acoord[1] < 0.5 and acoord[2] < 0.5 ):
        return psd.parts_eps[closest_node([coord[0],coord[1],coord[2]], psd.points)]
    else:
        return 1.0

def index2coord(index, size_arr, size_geo):
    index = (index/size_arr - 0.5)*size_geo
    return index

def vis(sim):
    sim.init_sim()
    eps_data = sim.get_epsilon()

    from mayavi import mlab
    s = mlab.contour3d(eps_data, colormap="YlGnBu")
    mlab.show()

Create the visualization for visual inspection

In [14]:
size_cell = [2, 2, 2]
size_solid = [1, 1, 1]
size_crystal_base = [0.1, 0.1, 0.1]
num_crystal = 190
np.random.seed(15)

param_geo_file_name = '../abaqus_working_space/abaqus_out/geometry.paramgeo'
num_geo_file_name = '../abaqus_working_space/abaqus_out/voronoi.geo'

pml_layers = [mp.PML(0.3)]

solid_region = mp.Block(size_solid, 
                    center = mp.Vector3(0, 0, 0),
                    material=mp.Medium(epsilon=10))
geometry = [solid_region,]

source_pad = 0.25
source = [mp.Source(mp.ContinuousSource(wavelength=2*(11**0.5), width=20),
                   component=mp.Ez,
                   center=mp.Vector3(0.75, 0, 0),
                   size=mp.Vector3(0, 0.1, 0.1))]
use_func = True

if use_func:
    sim = mp.Simulation(resolution=50,
                    cell_size=size_cell,
                    boundary_layers=pml_layers,
                    sources = source,
                    epsilon_func = my_eps)
else:
    sim = mp.Simulation(resolution=50,
                    cell_size=size_cell,
                    boundary_layers=pml_layers,
                    sources = source,
                    geometry=geometry)
vis(sim)

-----------
Initializing structure...
time for choose_chunkdivision = 0.00027895 s
Working in 3D dimensions.
Computational cell is 2 x 2 x 2 with resolution 50
subpixel-averaging is 19.7861% done, 16.7686 s remaining
subpixel-averaging is 26.7874% done, 11.4196 s remaining
subpixel-averaging is 33.1799% done, 8.13827 s remaining


In [36]:
# sim.run(mp.at_beginning(mp.output_epsilon), 
#        mp.to_appended("ez_small", mp.at_every(0.6, mp.output_efield_z)),
#        until=20)

one_cube_3d = []
def f(sim):
    one_cube_3d.append(sim.get_efield_z())    

sim.run(mp.at_every(1, f), until=200)
one_cube_3d = np.array(one_cube_3d)
one_cube_3d = one_cube_3d.transpose().astype('<f8')

with open('one_cube_3d_eps_10.bin', 'wb') as f:
    one_cube_3d.tofile(f)
# plt.imshow(ez_straight_waveg[1])

-----------
Initializing structure...
time for choose_chunkdivision = 0.000329971 s
Working in 3D dimensions.
Computational cell is 2 x 2 x 2 with resolution 50
     block, center = (0,0,0)
          size (1,1,1)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (10,10,10)
time for set_epsilon = 1.73558 s
-----------
Meep progress: 1.3800000000000001/200.0 = 0.7% done in 4.0s, 577.2s to go
on time step 138 (time=1.38), 0.0290598 s/step
Meep progress: 3.27/200.0 = 1.6% done in 8.0s, 482.0s to go
on time step 327 (time=3.27), 0.0211689 s/step
Meep progress: 5.28/200.0 = 2.6% done in 12.0s, 443.3s to go
on time step 528 (time=5.28), 0.0199453 s/step
Meep progress: 7.11/200.0 = 3.6% done in 16.0s, 434.8s to go
on time step 711 (time=7.11), 0.0218901 s/step
Meep progress: 9.06/200.0 = 4.5% done in 20.0s, 422.3s to go
on time step 906 (time=9.06), 0.0205534 s/step
Meep progress: 11.05/200.0 = 5.5% done in 24.0s, 411.2s to go
on time step 1105 (time=11.

Meep progress: 135.86/200.0 = 67.9% done in 276.8s, 130.7s to go
on time step 13586 (time=135.86), 0.0222155 s/step
Meep progress: 137.8/200.0 = 68.9% done in 280.8s, 126.8s to go
on time step 13780 (time=137.8), 0.0206715 s/step
Meep progress: 139.67000000000002/200.0 = 69.8% done in 284.8s, 123.0s to go
on time step 13967 (time=139.67), 0.0214412 s/step
Meep progress: 141.52/200.0 = 70.8% done in 288.9s, 119.4s to go
on time step 14152 (time=141.52), 0.0217159 s/step
Meep progress: 143.5/200.0 = 71.8% done in 292.9s, 115.3s to go
on time step 14350 (time=143.5), 0.0202981 s/step
Meep progress: 145.37/200.0 = 72.7% done in 296.9s, 111.6s to go
on time step 14537 (time=145.37), 0.0214753 s/step
Meep progress: 147.20000000000002/200.0 = 73.6% done in 300.9s, 107.9s to go
on time step 14720 (time=147.2), 0.0218996 s/step
Meep progress: 149.19/200.0 = 74.6% done in 304.9s, 103.8s to go
on time step 14919 (time=149.19), 0.0201798 s/step
Meep progress: 151.04/200.0 = 75.5% done in 308.9s, 1

Quick visulization of the generated geometry

In [33]:
# points = np.array([[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2],
#                    [2, 0], [2, 1], [2, 2]])
# from scipy.spatial import Voronoi, voronoi_plot_2d
# vor = Voronoi(points)
# import matplotlib.pyplot as plt
# voronoi_plot_2d(vor)
# plt.show()

# sim.init_sim()
# eps_data = sim.get_epsilon()

# ez_data = sim.get_array(center=mp.Vector3(), size=size_cell, component=mp.Ez)
# print(ez_data.shape)
# get_gif_out = False

# if get_gif_out:
#     !h5ls ez.h5
#     !mkdir temp
#     !mv eps-000000.00.h5 temp/eps-000000.00.h5
#     !mv ez.h5 temp/ez.h5
#     import os
#     os.chdir('temp')
#     !h5topng -z 20 -t 0:332 -R -Zc dkbluered -a yarg -A eps-000000.00.h5 ez.h5
#     !convert ez.t*.png ez.gif
#     os.chdir('..')

cheeky run of the geometry

In [34]:
one_cube_3d.shape

(100, 100, 100, 200)

visulization of the output field